# Dependencies

In [1]:
import requests
import logging
import boto3
from botocore.exceptions import ClientError
from base64 import b64encode
import pprint as pp
from xml.etree import ElementTree as ET
from requests_oauthlib import OAuth1
from requests_oauthlib import OAuth1Session

# API credentials
import api_config

# Vimeo setup
import vimeo

v = vimeo.VimeoClient(
    token=api_config.vimToken,
    key=api_config.vimClient,
    secret=api_config.vimSecret
)

C:\Users\mahlo\AppData\Roaming\Python\Python36\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.9) or chardet (2.0.3) doesn't match a supported version!
  RequestsDependencyWarning)


# Pull the videos down from TwentyThree

In [2]:
# OAuth setup
oauth = OAuth1Session(api_config.ttKey,
                      client_secret = api_config.ttConsumerSecret,
                      resource_owner_key = api_config.ttToken,
                      resource_owner_secret = api_config.ttTokenSecret)

In [24]:
# Get the count of total videos to download
response = oauth.get(f'{api_config.baseURL}api/photo/list')

targetElement = 0
iterator = 0


for i in response:
    if iterator == targetElement:
        i = str(i)
        splitEquals = i.split("=")
        lastElement = len(splitEquals) - 1
        splitQuote = splitEquals[lastElement].split('\"')
        totalRecords = splitQuote[1]
        print(f"Total records: {totalRecords}")
        
        break
    
    iterator += 1

Total records: 6


In [25]:
# Get the videos from 23
if int(totalRecords) < 100:
    totalRecords = 100
    
vidNum = 1
page = 1
videos = {}

print("----- Begin -----")
while page <= totalRecords/100:
    
    response = oauth.get(f'{api_config.baseURL}api/photo/list?size=100&p={page}')
#     response = requests.get(f'{api_config.baseURL}api/photo/list?size=100&p={page}', headers=headers)
    root = ET.fromstring(response.content)

    # Iterate over the XML response, and create a new object containing just the desired fields
    
    
    vidCount = len(root.getchildren())


    for child in root.iter('*'):
        # Clear the variables for this iteration
        ownerRaw = ''
        category = ''
        title = ''
        description = ''
        link = ''


        if child.tag == 'photo':
            print("---------------------------------------")
            print(f"Reviewing video #{vidNum} of {vidCount}")

            try:
                title = child.attrib['title']
                print(f"Title: {title}")
                
                description = child.attrib['title']
                print(f"Description: {description}")
            except:
                print("No title")
                print("No description")

            try:
                ownerRaw = child.attrib['username']
                print(f"Owner: {ownerRaw}")
            except:
                print("No User URL")

            if child.attrib['video_1080p_download'] != '':
                link = child.attrib['video_1080p_download']
                print(f"Download link: {link}")

            try:
                category = child.attrib['album_title']
                print(f"Category: {category}")
            except:
                print("No category")


            # Add the video information to the videos object
            videos[f'video_{vidNum}'] = {
                'Owner': ownerRaw,
                'Category': category,
                'Title': title,
                'Description': description,
                'Link': link
            }

            # Increment the video counter
            vidNum+=1


        elif child.tag == 'content_text':
            try:
                description = child.attrib['content_text']
                print(f"Description: {description}")
            except:
                continue

        else:
            continue


    page += 1
    print("---------------------------------------") 
    
print("----- Complete -----")

----- Begin -----
---------------------------------------
Reviewing video #1 of 6
Title: Introduction to the Gmail Add-On
Description: Introduction to the Gmail Add-On
Owner: mahlon.duke
Category: Internal
---------------------------------------
Reviewing video #2 of 6
Title: Introduction to the Gmail Add-On
Description: Introduction to the Gmail Add-On
Owner: mahlon.duke
Category: Internal
---------------------------------------
Reviewing video #3 of 6
Title: IT Engine + Accelo Case Study
Description: IT Engine + Accelo Case Study
Owner: taracallinan
Download link: /27288170/34873408/b2a31c11983a01cacb812707e8917f6c/video_1080p/it-engine-accelo-case-study-video.mp4
Category: Internal
---------------------------------------
Reviewing video #4 of 6
Title: Best Practices with the Projects Module
Description: Best Practices with the Projects Module
Owner: alyssaivancevich
Download link: /27288175/34872783/d6af5852f22a2b36ad8a8f11b96e78f1/video_1080p/best-practices-with-the-projects-module

In [26]:
# Create a list of unique video uploaders so that they can be mapped, and Categories to be created

def getUnique(videos):
    print("Getting unique data")

    uniqueOwners = []
    uniqueCategories = []

    for key, value in videos.items():
        #print(f"Key: {key}, Value: {value}")
        for key, value in value.items():
            if key == 'Owner':
                if value not in uniqueOwners:
                    uniqueOwners.append(value)

            if key == 'Category':
                if value not in uniqueCategories:
                    uniqueCategories.append(value)

    print(f'List of unique owners: {uniqueOwners}')
    print(f'List of unique categories: {uniqueCategories}')
    
    return(uniqueOwners, uniqueCategories)

In [27]:
# Function to map the owner

def mapOwners(videos):
    iterator = 1
    
    for vidNum, data in videos.items():
        video = vidNum
        owner = videos[video]['Owner']
        
        print(f"Iteration #{iterator}")
        print(f"First loop: {vidNum}, {data}")
        
#         print(f"Reviewing video # {video}")
#         print(f"Current owner: {owner}")
#         print(f"{vidNum} & {data}")

        
        for key, value in data.items():
            
            print(f"Second loop: {key}: {value}")
#             print("Now check owner \n")
            if key == 'Owner':
                if value == 'andrewgantan':
                    owner = 'Andrew Gantan'
                    print(f"New owner: {owner}")
                elif value == 'christaballingit':
                    owner = 'Christa Ballingit'
                    print(f"New owner: {owner}")
                elif value == 'coreyayers':
                    owner = 'Corey Ayers'
                    print(f"New owner: {owner}")
                elif value == 'alyssaivancevich':
                    owner = 'Alyssa Ivancevich'
                    print(f"New owner: {owner}")
                elif value == ('serenachan1' or 'serenachan'):
                    owner = 'Serena Chan'
                    print(f"New owner: {owner}")
                elif value =='peterescartin':
                    owner = 'Peter Escartin'
                    print(f"New owner: {owner}")
                
                    
                    
                else:
                    print("Owner not changed")
                    break
                    
                print("Iteration over")
                
        iterator += 1
                    
                    

    print(videos)
    
    return(videos)

In [11]:
uniqueOwners, uniqueCategories = getUnique(videos)
print(uniqueOwners)
print(uniqueCategories)

Getting unique data
List of unique owners: ['andrewgantan', 'christaballingit', 'coreyayers', 'alyssaivancevich', 'shunleisin', 'serenachan1', 'peterescartin', 'serenachan']
List of unique categories: ['Faces & Places of Accelo', 'Accelo Best Practices', 'Webinars', 'Accelo Help Videos', 'New User Videos: Intro to Accelo']
['andrewgantan', 'christaballingit', 'coreyayers', 'alyssaivancevich', 'shunleisin', 'serenachan1', 'peterescartin', 'serenachan']
['Faces & Places of Accelo', 'Accelo Best Practices', 'Webinars', 'Accelo Help Videos', 'New User Videos: Intro to Accelo']


In [12]:
videos = mapOwners(videos)
print(videos)

Iteration #1
First loop: video_1, {'Owner': 'andrewgantan', 'Category': 'Faces & Places of Accelo', 'Title': 'Faces & Places | Tasneem', 'Description': 'Faces & Places | Tasneem', 'Link': '/49543318/56300288/95eec35b4ddf80d9c3bb148d9581bcc9/video_1080p/faces-places-tasneem-video.mp4'}
Second loop: Owner: andrewgantan
New owner: Andrew Gantan
Iteration over
Second loop: Category: Faces & Places of Accelo
Second loop: Title: Faces & Places | Tasneem
Second loop: Description: Faces & Places | Tasneem
Second loop: Link: /49543318/56300288/95eec35b4ddf80d9c3bb148d9581bcc9/video_1080p/faces-places-tasneem-video.mp4
Iteration #2
First loop: video_2, {'Owner': 'andrewgantan', 'Category': 'Faces & Places of Accelo', 'Title': 'Molly Flynn - Faces of Accelo.mov', 'Description': 'Molly Flynn - Faces of Accelo.mov', 'Link': '/49543322/57216833/75b2ac70a611838e4181d257e3430e2b/video_1080p/molly-flynn-faces-of-accelomov-video.mp4'}
Second loop: Owner: andrewgantan
New owner: Andrew Gantan
Iteration o

# Vimeo

## Test the API Connection

In [29]:
# Make the request to the server for the "/me" endpoint.
about_me = v.get('/me')

# Make sure we got back a successful response.
assert about_me.status_code == 200

# Load the body's JSON data.
print(about_me.json())

{'uri': '/users/116142013', 'name': 'Mahlon Duke', 'link': 'https://vimeo.com/user116142013', 'location': '', 'gender': 'm', 'bio': None, 'short_bio': None, 'created_time': '2020-05-22T17:08:03+00:00', 'pictures': {'uri': '/users/116142013/pictures/40105027', 'active': True, 'type': 'custom', 'sizes': [{'width': 30, 'height': 30, 'link': 'https://i.vimeocdn.com/portrait/40105027_30x30'}, {'width': 75, 'height': 75, 'link': 'https://i.vimeocdn.com/portrait/40105027_75x75'}, {'width': 100, 'height': 100, 'link': 'https://i.vimeocdn.com/portrait/40105027_100x100'}, {'width': 300, 'height': 300, 'link': 'https://i.vimeocdn.com/portrait/40105027_300x300'}, {'width': 72, 'height': 72, 'link': 'https://i.vimeocdn.com/portrait/40105027_72x72'}, {'width': 144, 'height': 144, 'link': 'https://i.vimeocdn.com/portrait/40105027_144x144'}, {'width': 216, 'height': 216, 'link': 'https://i.vimeocdn.com/portrait/40105027_216x216'}, {'width': 288, 'height': 288, 'link': 'https://i.vimeocdn.com/portrait/

In [30]:
videos = {"vid1": 
          {"title": "The Title",
           "description": "A cool video",
           "Download Link": "google.com"
          },
         }

In [31]:
# Perform the upload
file_name = '{path_to_a_video_on_the_file_system}'
uri = client.upload(file_name, data={
  'name': 'Untitled',
  'description': 'The description goes here.'
})

print 'Your video URI is: %s' % (uri)

SyntaxError: invalid syntax (<ipython-input-31-47f77e54652a>, line 8)